In [12]:
import numpy as np
import pandas as pd

In [13]:
import torch
import torch.optim as optim
from sklearn import preprocessing
# For reproducibility
torch.manual_seed(1)

In [14]:
train_df = pd.read_csv("./자동차가격예측_data/train_data.csv")
test_df = pd.read_csv("./자동차가격예측_data/test_data.csv")
submit_df = pd.read_csv("./자동차가격예측_data/sample_submit.csv")
print(train_df.head())
print(test_df.head())
print(submit_df.head())

   ID  company    model  year  transmission  mileage  fueltype  tax   mpg  \
0   0        2    Focus  2019             1     3764         4  145  34.5   
1   1        1      I10  2015             1    27056         4   20  60.1   
2   2        3       A3  2019             3     3375         4  145  34.5   
3   3        3       A6  2019             0     5743         0  145  47.9   
4   4        2   Fiesta  2015             1    32782         4    0  65.7   

   engineSize  price  
0         2.3  26299  
1         1.0   5495  
2         2.0  32400  
3         2.0  29990  
4         1.0   7250  
   ID  company      model  year  transmission  mileage  fueltype  tax   mpg  \
0   0        2        Ka+  2017             1    10150         4  145  57.7   
1   1        2       Kuga  2013             1    50216         0  145  53.3   
2   2        3         Q5  2016             1    34189         0  200  47.9   
3   3        1        I10  2020             1     1900         4  145  56.5   
4   

In [15]:
## preprocessing LabelEncoding "Model" Column
le = preprocessing.LabelEncoder()
le.fit(train_df['model'])
train_df['model'] = le.transform(train_df['model'])
test_df['model'] = le.transform(test_df['model'])
print(le.classes_)

[' 1 Series' ' 2 Series' ' 3 Series' ' 4 Series' ' 5 Series' ' 6 Series'
 ' 7 Series' ' 8 Series' ' A1' ' A3' ' A4' ' A5' ' A6' ' A7' ' A8'
 ' Auris' ' Avensis' ' Aygo' ' B-MAX' ' C-HR' ' C-MAX' ' Camry' ' Corolla'
 ' EcoSport' ' Edge' ' Fiesta' ' Focus' ' Fusion' ' GT86' ' Galaxy'
 ' Getz' ' Grand C-MAX' ' Grand Tourneo Connect' ' Hilux' ' I10' ' I20'
 ' I30' ' I40' ' I800' ' IQ' ' IX20' ' IX35' ' Ioniq' ' KA' ' Ka+' ' Kona'
 ' Kuga' ' Land Cruiser' ' M2' ' M3' ' M4' ' M5' ' M6' ' Mondeo'
 ' Mustang' ' PROACE VERSO' ' Prius' ' Puma' ' Q2' ' Q3' ' Q5' ' Q7' ' Q8'
 ' R8' ' RAV4' ' RS3' ' RS4' ' RS5' ' RS6' ' S-MAX' ' S3' ' S4' ' S5'
 ' S8' ' SQ5' ' SQ7' ' Santa Fe' ' Supra' ' TT' ' Tourneo Connect'
 ' Tourneo Custom' ' Tucson' ' Urban Cruiser' ' Veloster' ' Verso'
 ' Verso-S' ' X1' ' X2' ' X3' ' X4' ' X5' ' X6' ' X7' ' Yaris' ' Z3' ' Z4'
 ' i3' ' i8']


In [16]:
x_train = np.array([train_df['company'],train_df['model'],train_df['year'],train_df['transmission'],train_df['mileage'],train_df['fueltype'],train_df['tax'],train_df['mpg'],train_df['engineSize']])
x_train = x_train.T
x_train = torch.FloatTensor(x_train).cuda()
y_train = torch.FloatTensor(train_df['price']).cuda()
print(x_train.shape)
print(y_train.shape)

torch.Size([17085, 9])
torch.Size([17085])


In [17]:
# 모델 초기화
w = torch.zeros((9,1), requires_grad=True,device="cuda")
b = torch.zeros(1, requires_grad=True,device="cuda")
# optimizer 설정
optimizer = optim.SGD([w, b], lr=2e-10)

nb_epochs = 2000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(w) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 500 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/2000 Cost: 383204160.000000
Epoch  500/2000 Cost: 160876432.000000
Epoch 1000/2000 Cost: 129498336.000000
Epoch 1500/2000 Cost: 114363248.000000
Epoch 2000/2000 Cost: 107060856.000000


In [18]:
x_test = torch.FloatTensor(np.array([test_df['company'],test_df['model'],test_df['year'],test_df['transmission'],test_df['mileage'],test_df['fueltype'],test_df['tax'],test_df['mpg'],test_df['engineSize']]).T).cuda()
y_test = x_test.matmul(w)+ b
print(y_test)

tensor([[13851.6221],
        [17478.1562],
        [16059.0273],
        ...,
        [17196.5977],
        [14249.2041],
        [20152.1230]], device='cuda:0', grad_fn=<AddBackward0>)


In [19]:
submit_df['price'] = y_test.cpu().detach().numpy()
submit_df.head()

,ID,price
0,0,13851.622070
1,1,17478.156250
2,2,16059.027344
3,3,13116.944336
4,4,20129.070312


In [20]:
submit_df.to_csv("./배추가격예측_data/submit_sample.csv",index=False)